In [82]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

In [83]:
Train_data = pd.read_csv(r"C:\Users\tiwar\Downloads\Train_Data.csv")

In [84]:
Test_data = pd.read_csv(r"C:\Users\tiwar\Downloads\Test_Data.csv")

In [85]:
Sub_sample= pd.read_csv(r"C:\Users\tiwar\Downloads\Sample_Submission.csv")

In [86]:
Train_data.head()

,duration,protocoltype,service,flag,srcbytes,dstbytes,land,wrongfragment,urgent,hot,...,dsthostsamesrvrate,dsthostdiffsrvrate,dsthostsamesrcportrate,dsthostsrvdiffhostrate,dsthostserrorrate,dsthostsrvserrorrate,dsthostrerrorrate,dsthostsrvrerrorrate,lastflag,attack
0,0,tcp,netbios_dgm,REJ,0,0,0,0,0,0,...,0.06,0.06,0.00,0.00,0.00,0.0,1.00,1.0,21,1
1,0,tcp,smtp,SF,1239,400,0,0,0,0,...,0.45,0.04,0.00,0.00,0.11,0.0,0.02,0.0,18,0
2,0,tcp,http,SF,222,945,0,0,0,0,...,1.00,0.00,0.02,0.03,0.00,0.0,0.00,0.0,21,0
3,0,tcp,http,SF,235,1380,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,21,0
4,0,tcp,uucp_path,REJ,0,0,0,0,0,0,...,0.01,0.08,0.00,0.00,0.00,0.0,1.00,1.0,19,1


In [87]:
Train_data.info

<bound method DataFrame.info of        duration protocoltype      service flag  srcbytes  dstbytes  land  \
0             0          tcp  netbios_dgm  REJ         0         0     0   
1             0          tcp         smtp   SF      1239       400     0   
2             0          tcp         http   SF       222       945     0   
3             0          tcp         http   SF       235      1380     0   
4             0          tcp    uucp_path  REJ         0         0     0   
...         ...          ...          ...  ...       ...       ...   ...   
86840         0          tcp         http   SF       216       674     0   
86841         0          tcp         http   SF       368      1725     0   
86842        28          tcp          ftp   SF      1483      4152     0   
86843         0          tcp         smtp   SF      1679       385     0   
86844     11343          tcp       telnet   SF      3396     16694     0   

       wrongfragment  urgent  hot  ...  dsthostsamesrvr

In [88]:
Train_data.shape

(86845, 43)

In [89]:
Train_data['protocoltype'].value_counts()

protocoltype
tcp     75876
udp      9929
icmp     1040
Name: count, dtype: int64

In [90]:
Train_data['flag'].value_counts()

flag
SF      50672
S0      27773
REJ      6721
RSTO     1127
S1        294
RSTR      106
S2        103
S3         38
OTH        10
SH          1
Name: count, dtype: int64

In [91]:
Train_data['service'].value_counts()

service
http        31505
private     12859
domain_u     7196
smtp         5806
ftp_data     4929
            ...  
printer        46
urh_i           9
red_i           6
tim_i           4
tftp_u          3
Name: count, Length: 65, dtype: int64

In [92]:
Train_data= pd.get_dummies(Train_data, columns=['protocoltype'])
lable_encoder= LabelEncoder()
Train_data['flag']= lable_encoder.fit_transform(Train_data['flag'])
Train_data['service']= lable_encoder.fit_transform(Train_data['service'])

In [93]:
Train_data.head()

,duration,service,flag,srcbytes,dstbytes,land,wrongfragment,urgent,hot,numfailedlogins,...,dsthostsrvdiffhostrate,dsthostserrorrate,dsthostsrvserrorrate,dsthostrerrorrate,dsthostsrvrerrorrate,lastflag,attack,protocoltype_icmp,protocoltype_tcp,protocoltype_udp
0,0,33,1,0,0,0,0,0,0,0,...,0.00,0.00,0.0,1.00,1.0,21,1,False,True,False
1,0,49,8,1239,400,0,0,0,0,0,...,0.00,0.11,0.0,0.02,0.0,18,0,False,True,False
2,0,22,8,222,945,0,0,0,0,0,...,0.03,0.00,0.0,0.00,0.0,21,0,False,True,False
3,0,22,8,235,1380,0,0,0,0,0,...,0.00,0.00,0.0,0.00,0.0,21,0,False,True,False
4,0,62,1,0,0,0,0,0,0,0,...,0.00,0.00,0.0,1.00,1.0,19,1,False,True,False


In [94]:
Train_feat= Train_data.drop(columns=['attack'])
Train_targ = Train_data['attack']

In [95]:
knn  = KNeighborsClassifier()
param_grid = {'n_neighbors': range(1, 21)}
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')
grid_search.fit(Train_feat, Train_targ)

best_k = grid_search.best_params_['n_neighbors']
print(f'Best n_neighbors: {best_k}')

Best n_neighbors: 2


In [96]:
log_reg= LogisticRegression(solver='saga', max_iter=5000, tol=1e-3)
knn = KNeighborsClassifier(n_neighbors=best_k)
dt= DecisionTreeClassifier()

In [97]:
voting_clf= VotingClassifier(estimators=[('lr', log_reg), ('knn', knn), ('dt', dt)] , voting='hard')
voting_clf.fit(Train_feat, Train_targ)

Train_preddction =voting_clf.predict(Train_feat)
Train_accuracy= accuracy_score(Train_targ, Train_preddction)
print(Train_accuracy)

1.0


In [98]:
Test_data= pd.get_dummies(Test_data, columns=['protocoltype'])
lable_encoder= LabelEncoder()
Test_data['flag']= lable_encoder.fit_transform(Test_data['flag'])
Test_data['service']= lable_encoder.fit_transform(Test_data['service'])

In [99]:
Test_prediction= voting_clf.predict(Test_data)

In [100]:
Final_sub= Sub_sample.copy()
Final_sub['attack']=Test_prediction
Final_sub.to_csv('Final_sub.csv' , index ='False')